In [1]:
import torch
import torch.nn as nn

In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [3]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [4]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method.

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [6]:
class TransformerBlock( nn.Module ):
    def __init__( self, cfg ):
        super().__init__()
        self.norm1 = LayerNorm( cfg["emb_dim"] )
        self.att = MultiHeadAttention( 
            d_in = cfg["emb_dim"], 
            d_out = cfg["emb_dim"], 
            context_length = cfg["context_length"], 
            dropout = cfg["drop_rate"], 
            num_heads = cfg["n_heads"] 
        )
        self.dropout = nn.Dropout( cfg["drop_rate"] )
        self.norm2 = LayerNorm( cfg["emb_dim"] )
        self.feedforward = FeedForward( cfg )
        
    def forward( self, x ):
        
        # the attention chunk in this transformer:
        shortcut = x 
        x = self.norm1( x )
        x = self.att( x )
        x = self.dropout( x )
        x = x + shortcut 
        
        # the feed forward chunk:
        shortcut = x
        x = self.norm2( x ) 
        x = self.feedforward( x )
        x = self.dropout( x )
        x = x + shortcut 
        
        return x 

In [7]:
# instantiate TransformerBlock
block = TransformerBlock( GPT_CONFIG_124M )

In [8]:
input = torch.randn( 3, 4, 768 )

In [9]:
input

tensor([[[ 5.5222e-01, -5.4585e-01, -4.4539e-01,  ...,  2.2030e-02,
          -1.0146e+00,  1.3088e-01],
         [ 2.8190e+00, -5.6559e-01, -1.2963e+00,  ...,  1.4356e+00,
           1.6732e+00, -6.0555e-01],
         [ 1.6607e+00,  4.8229e-01,  9.8355e-02,  ..., -2.0122e+00,
           9.5335e-01,  6.7414e-01],
         [-6.7907e-01, -2.1143e+00,  5.9474e-01,  ...,  3.7531e-01,
          -2.7407e-01,  4.4141e-01]],

        [[ 5.2995e-01,  2.9010e-02, -9.5663e-01,  ...,  5.2055e-01,
           1.2935e+00,  1.4557e-01],
         [-5.9731e-06, -2.1768e-01, -2.6063e-01,  ...,  3.3984e-01,
           1.0078e+00, -2.3623e-01],
         [ 1.6111e+00, -9.4355e-01, -1.1979e+00,  ..., -1.3324e+00,
          -5.5246e-01, -4.4921e-01],
         [-3.2227e-01, -5.8485e-01, -9.6701e-01,  ..., -7.4933e-01,
           1.4700e+00,  1.5574e+00]],

        [[-9.8341e-01, -3.6800e-01, -7.1527e-01,  ..., -3.9969e-01,
          -3.2310e-01, -5.4688e-01],
         [ 1.1630e+00, -3.4938e-01, -1.0327e+00,  .

In [10]:
output = block( input )

In [11]:
output.shape

torch.Size([3, 4, 768])

In [12]:
output

tensor([[[ 0.6439, -0.3974, -1.0420,  ...,  0.6021, -1.6177,  0.3859],
         [ 2.6685, -0.6266, -1.2688,  ...,  1.4672,  1.7759, -0.9086],
         [ 1.4939,  0.7233, -0.5906,  ..., -1.8612,  0.5093,  1.1246],
         [-0.9961, -2.0111,  0.6847,  ...,  0.2212, -0.3329,  0.8191]],

        [[ 0.6004, -0.3200, -1.2392,  ...,  0.7127,  1.3571,  0.1922],
         [ 0.3905, -0.2807, -0.4481,  ...,  0.4241,  0.5491, -0.7410],
         [ 1.5182, -0.8880, -0.7294,  ..., -1.4782, -0.7820, -0.5405],
         [-0.4605, -0.5626, -0.6870,  ..., -0.9831,  1.2699,  1.3428]],

        [[-0.6186, -0.4320, -0.9639,  ..., -0.4070, -0.4752,  0.2853],
         [ 1.2175,  0.0773, -1.2669,  ..., -1.3302, -0.4204, -2.8902],
         [-0.8694, -0.5681,  0.5368,  ..., -0.2580,  0.8459, -0.5935],
         [ 1.8736, -1.4744, -0.3151,  ..., -0.0278, -2.1050, -0.5818]]],
       grad_fn=<AddBackward0>)